In [1]:
import random

In [2]:
from firebase_admin import initialize_app, credentials, db

In [3]:
cred = credentials.Certificate('creds.json')

In [4]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [15]:
H = "Holz"
L = "Lehm"
W = "Weizen"
S = "Schafe"
E = "Erz"

class Game:
    
    resources = [H, L, W, S, E]
    
    def __init__(self, aliases):
        
        self.turn_ref = db.reference("/turn")
        self.board_ref = db.reference("/board")
        self.stack_ref = db.reference("/stack")
        self.player_refs = [db.reference('/players/' + a) for a in aliases]
        self.player_count = len(self.player_refs)
        self.resource_order = {r: i for i, r in enumerate(self.resources)}
        
        
    def set_board(self, board):
        
        self.board_ref.set(board)
        
    
    def initialize(self):
        
        for player_num, player_ref in enumerate(self.player_refs):
            player_ref.set({
                "number": player_num,
                "resources": [],
                "cards": [],
            })
        
        stack = ["Ritter"] * 14 + ["Monopol", "Strassenbau", "Erfindung"] * 2
        stack += ["Bibliothek", "Marktplatz", "Rathaus", "Kirche", "Universität"]
        random.shuffle(stack)
        self.stack_ref.set(stack)
        
        first = random.randint(0, self.player_count - 1)
        self.turn_ref.set({'player': first, 'maxKnightCount': 2})
        self.set_turn(first, None)
        
    
    def set_turn(self, player_num, roll):
        
        self.current = player_num
        turn = self.turn_ref.get()
        turn['player'] = self.current
        turn['roll'] = roll
        self.turn_ref.set(turn)
        
        
    def set_longest_road(self, player_num, roads):
        
        turn = self.turn_ref.get()
        turn['longestRoadPlayer'] = player_num
        turn['longestRoadCount'] = roads
        self.turn_ref.set(turn)
        
        
    def next_turn(self, roll, step=1):
        
        next_player = (self.current + step) % self.player_count
        self.set_turn(next_player, roll)
        
        
    def next_roll(self):
        
        roll = random.randint(1, 6) + random.randint(1, 6)
        self.next_turn(roll)
        
        
    def update_player(self, player_num, gains=None, losses=None, gained_cards=None, lost_cards=None):
        
        player = self.player_refs[player_num].get()
        
        if losses is not None:
            for resource in losses:
                player['resources'].remove(resource)
        if gains is not None:
            player['resources'] += gains
        player['resources'] = sorted(player['resources'], key=lambda r: self.resource_order[r])
        
        if lost_cards is not None:
            for card in lost_cards:
                player['cards'].remove(card)
        if gained_cards is not None:
            player['cards'] += gained_cards
            if "Ritter (gespielt)" in gained_cards:
                knight_count = player['cards'].count("Ritter (gespielt)")
                turn = self.turn_ref.get()
                if knight_count > turn['maxKnightPlayer']:
                    turn['maxKnightCount'] = knight_count
                    turn['maxKnightPlayer'] = player_num
                    self.turn_ref.set(turn)
            
        self.player_refs[player_num].set(player)
            
            
    def trade_resources(self, giver_num, taker_num, give, take):
        
        self.update_player(giver_num, gains=take, losses=give)
        self.update_player(taker_num, gains=give, losses=take)
        
        
    def steal_resource(self, dest_num, orig_num):
        
        orig = self.player_refs[orig_num].get()
        if len(orig['resources']) == 0:
            return
        resource = random.choice(orig['resources'])
        self.update_player(orig_num, losses=[resource])
        self.update_player(dest_num, gains=[resource])
            
            
    def drop_resources(self, player_num):
                
        player = self.player_refs[player_num].get()
        
        over_count = len(player['resources']) - 7
        if over_count < 1:
            return
        
        drops = random.sample(player['resources'], over_count)
        self.update_player(player_num, losses=drops)
        
        
    def buy(self, player_num, roads=0, towns=0, cities=0, cards=0):
        
        costs = [H, L] * roads + [H, L, W, S] * towns + [W, W, E, E, E] * cities + [W, S, E] * cards
        
        drawn_cards = None
        if cards > 0:
            stack = self.stack_ref.get()
            drawn_cards = stack[:cards]
            self.stack_ref.set(stack[cards:])
        
        self.update_player(player_num, losses=costs, gained_cards=drawn_cards)
        
        
    def play_card(self, player_num, card, param=None):
        
        gains = None
        gained_cards = None
        
        if card == 'Ritter':
            gained_cards = ["Ritter (gespielt)"]
                
        elif card == 'Strassenbau':
            # TODO: grant 2 streets?
            pass
            
        elif card == 'Monopol':
            
            gains = []
            for other_num, player_ref in enumerate(self.player_refs):
                if other_num == player_num:
                    continue
                other = self.player_refs[other_num].get()
                losses = [r for r in other['resources'] if r == param]
                gains += losses
                self.update_player(other_num, losses=losses)
                
        elif card == 'Erfindung':
            
            gains = params
            
        else:
            return
            
        self.update_player(player_num, gains=param, gained_cars=gained_cards, lost_cards=[card])
        
        

# Init

In [16]:
game = Game(['erns', 'nigh'])

In [9]:
game.set_board([    
"                                 x           ",
"                              + = +          ",
"                            /     |          ",
"                   e  + - +   W   + - +      ",
"                    %     |   4 /     $ s    ",
"              + - +   L   + - +   H   + - +  ",
"            /     |   2 /     |   6 /     |  ",
"          +   S   + - +   H   + - +   W   +  ",
"          |   9 /     |   5 /     |   9 /    ",
"       w  + - +   L   + - +   S   + - +      ",
"        %     |   8 /  *  |  12 /     $ x    ",
"      +   H   + - +   X   + - +   S   +      ",
"      |   3 /     |     /     |   4 /        ",
"      + - +   E   + - +   E   + - +          ",
"    /     |   5 /     |   8 /     |          ",
"  +   W   + - +   L   + - +   S   +          ",
"x $   3 /     |  10 /     |  10 %            ",
"  + - +   W   + - +   H   + - +  x           ",
"      |   6 /     |  11 /                    ",
"      + = +   E   + = +                      ",
"       h  |  11 /  l                         ",
"          + - +                              ",
"                                             "
])

# Play

In [17]:
game.initialize()

In [ ]:
game.next_turn(None, 1)

In [ ]:
game.next_turn(None, -1)

In [ ]:
game.roll_next()

In [ ]:
game.draw_card(0)

In [ ]:
game.play_card(0, "Ritter")

In [ ]:
game.steal_resource(0, 1)

In [ ]:
game.drop_resources(0)

In [ ]:
game.buy(0, roads=2)

In [ ]:
game.set_longest_road(0, 5)

In [ ]:
game.set_board([    
"                                 x           ",
"                              + = +          ",
"                            /     |          ",
"                   e  + -2+   W   + - +      ",
"                    %     |2  4 /     $ s    ",
"              + - +   L   + -2#2  H   + - +  ",
"            /     |   2 /     |   6 /     |  ",
"          +   S   + -0+0  H   + - +1  W   +  ",
"          |   9 /     |   5 /     |1  9 /    ",
"       w  + - +   L   + - +   S   + -1+1     ",
"        %     |   8 /     |  12 /1    $ x    ",
"      +   H   +0- +   X   + - +1  S   +      ",
"      |   3 /0    |     /     |1  4 /        ",
"      + - +   E   + - +   E   + - +          ",
"    /     |0  5 /     |   8 /1    |          ",
"  +   W   + - +2  L   #1- +   S   +          ",
"x $   3 /0 *  |2 10 /1    |  10 %            ",
"  + - +0  W   + - +   H   + - +  x           ",
"      |   6 /2    |  11 /                    ",
"      + = +2  E   + = +                      ",
"       h  |  11 /  l                         ",
"          + - +                              ",
"                                             "
])